# Data exploration

In [9]:
from datasets import load_dataset
from transformers import pipeline
from tqdm import tqdm
from transformers import AutoTokenizer
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
from transformers import DistilBertTokenizerFast
import time
import numpy as np
import pandas as pd
import torch

In [10]:
split = "train"
cache_dir = "./data_cache"

dialogue_dataset = load_dataset(
    "doc2dial",
    name="dialogue_domain",  # this is the name of the dataset for the second subtask, dialog generation
    split=split,
    ignore_verifications=True,
    cache_dir=cache_dir,
)

Reusing dataset doc2dial (./data_cache/doc2dial/dialogue_domain/1.0.1/c15afdf53780a8d6ebea7aec05384432195b356f879aa53a4ee39b740d520642)


In [11]:
document_dataset = load_dataset(
    "doc2dial",
    name="document_domain",  # this is the name of the dataset for the second subtask, dialog generation
    split=split,
    ignore_verifications=True,
    cache_dir=cache_dir,
)

Reusing dataset doc2dial (./data_cache/doc2dial/document_domain/1.0.1/c15afdf53780a8d6ebea7aec05384432195b356f879aa53a4ee39b740d520642)


In [12]:
dialogue_dataset[0]

{'dial_id': '9f44c1539efe6f7e79b02eb1b413aa43',
 'doc_id': 'Top 5 DMV Mistakes and How to Avoid Them#3_0',
 'domain': 'dmv',
 'turns': [{'turn_id': 1,
   'role': 'user',
   'da': 'query_condition',
   'references': [{'sp_id': '4', 'label': 'precondition'}],
   'utterance': 'Hello, I forgot o update my address, can you help me with that?'},
  {'turn_id': 2,
   'role': 'agent',
   'da': 'respond_solution',
   'references': [{'sp_id': '6', 'label': 'solution'},
    {'sp_id': '7', 'label': 'solution'}],
   'utterance': 'hi, you have to report any change of address to DMV within 10 days after moving. You should do this both for the address associated with your license and all the addresses associated with all your vehicles.'},
  {'turn_id': 3,
   'role': 'user',
   'da': 'query_solution',
   'references': [{'sp_id': '56', 'label': 'solution'}],
   'utterance': 'Can I do my DMV transactions online?'},
  {'turn_id': 4,
   'role': 'agent',
   'da': 'respond_solution',
   'references': [{'sp_id

In [13]:
document_dataset[250]

{'domain': 'dmv',
 'doc_id': 'Top 5 DMV Mistakes and How to Avoid Them#3_0',
 'title': 'Top 5 DMV Mistakes and How to Avoid Them#3',
 'doc_text': 'Many DMV customers make easily avoidable mistakes that cause them significant problems, including encounters with law enforcement and impounded vehicles. Because we see customers make these mistakes over and over again , we are issuing this list of the top five DMV mistakes and how to avoid them. \n\n1. Forgetting to Update Address \nBy statute , you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. It is not sufficient to only: write your new address on the back of your old license; tell the United States Postal Service; or inform the police officer writing you a ticket. If you fail to keep your address current , you will miss a suspension order and may be charged with operati

In [14]:
search_domain = 'dmv'
search_doc_id = 'Top 5 DMV Mistakes and How to Avoid Them#3_0'
search_id_sp = ['6', '7']

def text_from_spans(search_domain, search_doc_id, search_id_sp, document_dataset):
    start = time.time()
    total_answer = ''
    for doc in document_dataset:
        if doc['domain'] == search_domain and doc['doc_id'] == search_doc_id:
            for span in doc['spans']:
                if span['id_sp'] in search_id_sp:
                    total_answer+=span['text_sp']
            break
    print(f"Time elapsed: {time.time() - start}")
    return total_answer

text_from_spans(search_domain, search_doc_id, search_id_sp, document_dataset)

Time elapsed: 0.2763059139251709


'you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. '

In [165]:
question_answerer = pipeline("question-answering")

No model was supplied, defaulted to distilbert-base-cased-distilled-squad (https://huggingface.co/distilbert-base-cased-distilled-squad)


In [166]:
context = "By statute , you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ."

In [188]:
result = question_answerer(question="Hello, I forgot o update my address, can you help me with that?", context=context)
print(result)

{'score': 0.08009324222803116, 'start': 137, 'end': 144, 'answer': 'license'}


## Creating the dataset

Steps:
- [X] Sliding windows from the Document
- [ ] Extract user utterance
- [ ] Extract Dialogue history

### Sliding windows from the Document

In [15]:
# Tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('bert-base-uncased')

# Defining train_dict
train_dict = dict()
train_dict['train_document'] = []
train_dict['train_id_sp'] = []
train_dict['train_user_utterance'] = []
train_dict['train_doc_domain'] = []
train_dict['train_doc_id'] = []
train_dict['train_text_sp'] = []
train_dict['train_dial_id_turn_id'] = []     # necessary for evaluation
train_dict['train_start_pos'] = []     
train_dict['train_end_pos'] = []     
train_dict['train_start_tok'] = []     
train_dict['train_end_tok'] = []  

start = time.time()
for idx, dialogue in tqdm(enumerate(dialogue_dataset)):
    if idx == 10:
        break
    dial_id_turn_id = []       # running list of <dial_id>_<turn_id> for evaluation
    sp_id_list = []            # running list of spans per document
    user_utterance_list = []   # running list of user utterances per document
    
    for turn in dialogue['turns']:
        dial_id_turn_id.append(dialogue['dial_id'] + '_' + str(turn['turn_id']))
        if turn['role'] == 'user':
            # TURN UTTERANCE IS FLATTENED AND ONLY THE [INPUT_IDS] IS STORED
            turn['utterance'] = tokenizer(turn['utterance'], padding=True, truncation=True, return_tensors="pt")['input_ids'].view(-1)
            user_utterance_list.append(turn['utterance'])   # adding user utterance to user_utterance_list
        else:
            references = turn['references']
            ref_sp_id = []
            for ref in references:
                ref_sp_id.append(ref['sp_id'])
            sp_id_list.append(ref_sp_id)          # adding list of sp_ids per dialogue to list of sp_ids per document
    train_dict['train_id_sp'].append(sp_id_list)
    train_dict['train_user_utterance'].append(user_utterance_list)
    train_dict['train_doc_domain'].append(dialogue['domain'])
    train_dict['train_doc_id'].append(dialogue['doc_id'])
    train_dict['train_dial_id_turn_id'].append(dial_id_turn_id)
    
    for doc in document_dataset:
        if doc['doc_id'] == train_dict['train_doc_id'][-1]:
            # DOCUMENT TEXT IS NOT A TENSOR. PREVIOUSLY WE HAD tokenizer( )['index_ids'].view(-1)
            doc['doc_text'] = tokenizer(doc['doc_text'], padding=True, truncation=False, return_tensors="pt")
            train_dict['train_document'].append(doc['doc_text'])          # adding the total document text
            text_sp_2 = []            
            start_sp_list = []         # big start sp list
            end_sp_list = []           # big end sp list        
            start_tok_list = []         # big start token list
            end_tok_list = []           # big end token list     
            for train_spans_id in train_dict['train_id_sp'][-1]:    
                text_sp = ""         
                ref_start_pos_list = []
                ref_end_pos_list = []      
                for span in doc['spans']:                    
                    if span['id_sp'] in train_spans_id:
                        text_sp += span['text_sp']                        
                        ref_start_pos_list.append(span['start_sp'])
                        ref_end_pos_list.append(span['end_sp'])    
                start_pos = np.amin(ref_start_pos_list)
                start_sp_list.append(start_pos)
                # convert start_pos to start_token
                start_tok_pos = doc['doc_text'].char_to_token(start_pos)
                start_tok_list.append(start_tok_pos)
                # convert end_pos to end_token
                end_pos = np.amax(ref_end_pos_list)
                end_sp_list.append(end_pos)
                end_tok_pos = doc['doc_text'].char_to_token(end_pos)
                end_tok_list.append(end_tok_pos)
                text_sp_2.append(text_sp)
            train_dict['train_text_sp'].append(text_sp_2)
            train_dict['train_start_pos'].append(start_sp_list)
            train_dict['train_end_pos'].append(end_sp_list)
            train_dict['train_start_tok'].append(start_tok_list)
            train_dict['train_end_tok'].append(end_tok_list)
            break
end = time.time()
print(f'Total time: {end-start}')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DistilBertTokenizerFast'.
0it [00:00, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
10it [00:01,  5.09it/s]

Total time: 1.9763898849487305


Results:

In [16]:
print('User utterances:')
print(train_dict['train_user_utterance'][0])

print('\nID Sp:')
print(train_dict['train_id_sp'][0])

print('\nDoc ID:')
print(train_dict['train_doc_id'][0])

print('\nDoc domain:')
print(train_dict['train_doc_domain'][0])

print('\nTrain text spans:')
print(train_dict['train_text_sp'][0])

print('\nDial_ID Turn_ID:')
print(train_dict['train_dial_id_turn_id'][0])

User utterances:
[tensor([  101,  7592,  1010,  1045,  9471,  1051, 10651,  2026,  4769,  1010,
         2064,  2017,  2393,  2033,  2007,  2008,  1029,   102]), tensor([  101,  2064,  1045,  2079,  2026,  1040,  2213,  2615, 11817,  3784,
         1029,   102]), tensor([  101,  4283,  1010,  1998,  1999,  2553,  1045,  5293,  2000,  3288,
         2035,  1997,  1996, 12653,  2734,  2000,  1996,  1040,  2213,  2615,
         2436,  1010,  2054,  2064,  1045,  2079,  1029,   102]), tensor([ 101, 7929, 1010, 1998, 2064, 2017, 2425, 2033, 2153, 2073, 2323, 1045,
        3189, 2026, 2047, 4769, 1029,  102]), tensor([ 101, 2064, 2017, 2425, 2033, 2062, 2055, 4026, 2685, 1998, 2037, 3465,
        1029,  102])]

ID Sp:
[['6', '7'], ['56'], ['49', '50', '52'], ['6', '7'], ['41', '43']]

Doc ID:
Top 5 DMV Mistakes and How to Avoid Them#3_0

Doc domain:
dmv

Train text spans:
['you must report a change of address to DMV within ten days of moving. That is the case for the address associated with 

In [12]:
print('\nDoc text:')
print(train_dict['train_document'][0])


Doc text:
tensor([  101,  2408,   141,  2107,  2559,  5793,  1294,  3253,  3644,  1895,
        12572,  1115,  2612,  1172,  2418,  2645,   117,  1259, 12270,  1114,
         1644,  7742,  1105, 24034,  6094, 21160,  4011,   119,  2279,  1195,
         1267,  5793,  1294,  1292, 12572,  1166,  1105,  1166,  1254,   117,
         1195,  1132, 16141,  1142,  2190,  1104,  1103,  1499,  1421,   141,
         2107,  2559, 12572,  1105,  1293,  1106,  3644,  1172,   119,   122,
          119, 17323,  1916,  1106,  3725,  9216, 24930, 18380,  1650, 14610,
          117,  1128,  1538,  2592,   170,  1849,  1104,  4134,  1106,   141,
         2107,  2559,  1439,  1995,  1552,  1104,  2232,   119,  1337,  1110,
         1103,  1692,  1111,  1103,  4134,  2628,  1114,  1240,  5941,   117,
         1112,  1218,  1112,  1155,  1103, 11869,  2628,  1114,  1296,  4410,
         3686,   117,  1134,  1336, 11271,   119,  1135,  1110,  1136,  6664,
         1106,  1178,   131,  3593,  1240,  1207,  41

## Create a Dataframe out of the train_dictionary

In [13]:
data = pd.DataFrame(train_dict)

In [ ]:
from datasets import load_metric

metric = load_metric("squad_v2")
print(metric.features) #this shows you what format the metric is expecting

prediction = {'id': <rc dataset is of shape dialid_turnid - this value has to match the answer>,
              'prediction_text': <your prediction>,
              'no_answer_probability': 0.0} #edwin said we can ignore this for task 1
reference = {'id': <see prediction>, 
              'answers': {
                  'text': [list of answer, best to use the ones from the rc dataset],                                       
                  'answer_start': [list of numbers of the answer star char again see rc dataset. ]}
            }

metric.add(prediction=prediction, reference=reference)
final_score = metric.compute()
final_score

# Testing model

In [17]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [44]:
question=train_dict['train_user_utterance'][0][0]
print(f'Decoded question: {tokenizer.decode(question)}')
# If already tokenized from dataset
text=train_dict['train_document'][0]    # tokenized text
# if simple text
#text='By statute , you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ.'
#text=tokenizer([text],  return_tensors="pt")['input_ids'].view(-1)

input_ids=torch.cat((question, text['input_ids'].view(-1)[1:]), 0)
SEP_token_id=102
sep_idx = (input_ids == 102).nonzero(as_tuple=False)[0][0].item()
num_seg_a = sep_idx+1
num_seg_b = len(input_ids) - num_seg_a
segment_ids = [0]*num_seg_a + [1]*num_seg_b
assert len(segment_ids) == len(input_ids)


Decoded question: [CLS] hello, i forgot o update my address, can you help me with that? [SEP]


Create mask for start and end positions. This way we only check the first token after the '.' as start positions, and the tokens before the '.' as end positions.

In [45]:
# token id for '.' = 1012
def mask_start_end(input_ids_trunc, segment_ids_trunc, mode):
    """Returns a mask for the start and end logits. 
    input_ids_trunc = tokens (tensor)
    segment_ids_trunc = mask (question / text)
    mode = "start" or 'end'
    return tensor
    """
    a = torch.where(input_ids_trunc == 1012, 1, 0)   # mask=1 for '.'
    a = a * torch.tensor(segment_ids_trunc)          # mask question - text
    if mode=='start':
        b = torch.cat((torch.tensor([0]),a),0)[:-1]     # move the 1s one position to the right
    else:
        b = torch.cat((a, torch.tensor([0])),0)[1:]
    assert len (a) == len(b)
    return b

def tensor_to_positive(tensor, mask):
    """ All the values need to be higher than 0, since 0s are values for the mask
    and we don't want to choose them when selecting the start or end token.
    Return torch.tensor """
    min_value = torch.amin(tensor) 
    tensor_positive = tensor + (mask * np.abs(min_value.detach().numpy()))
    return tensor_positive

In [46]:
input_ids_trunc = input_ids[:511]
input_ids_trunc = torch.cat((input_ids_trunc, torch.tensor([102])),0)
segment_ids_trunc = segment_ids[:512]

output = model(input_ids_trunc.view(1,-1), token_type_ids=torch.tensor([segment_ids_trunc]))
tokens = tokenizer.convert_ids_to_tokens(input_ids_trunc)   # table with token_id -> word
#tokens with highest start and end scores
mask_start = mask_start_end(input_ids_trunc, segment_ids_trunc, 'start')
start_logits_positive = tensor_to_positive(output.start_logits * mask_start, mask_start)
answer_start = torch.argmax(start_logits_positive)  # token index for the highest start token
mask_end = mask_start_end(input_ids_trunc, segment_ids_trunc, 'end')
end_logits_positive = tensor_to_positive(output.end_logits * mask_end, mask_end)
answer_end = torch.argmax(end_logits_positive)
if answer_end >= answer_start:
    answer = " ".join(tokens[answer_start:answer_end+1])
else:
    print("I am unable to find the answer to this question. Can you please ask another question?")
    
print("\nQuestion:\n{}".format(tokenizer.decode(question)))
print("\nAnswer:\n{}.".format(answer))


Question:
[CLS] hello, i forgot o update my address, can you help me with that? [SEP]

Answer:
forgetting to update address by statute , you must report a change of address to d ##m ##v within ten days of moving.


### Output - For Report
This section of text shows that span [49][50][51][52] is what we return. However, the section belows is what the ground truth says. SPan [51] is highlighted in red (we return 51, ground truth doesn't contain it).

- 'About ten percent of customers visiting a DMV office do not bring what they need to complete their transaction, and have to come back a second time to finish their business. This can be as simple as not bringing sufficient funds to pay for a license renewal or not having the proof of auto insurance required to register a car. <font color='red'>Better yet ,</font> don t visit a DMV office at all, and see if your transaction can be performed online, like an address change, registration renewal, license renewal, replacing a lost title, paying a DRA or scheduling a road test. '
- 'About ten percent of customers visiting a DMV office do not bring what they need to complete their transaction, and have to come back a second time to finish their business. This can be as simple as not bringing sufficient funds to pay for a license renewal or not having the proof of auto insurance required to register a car. don t visit a DMV office at all, and see if your transaction can be performed online, like an address change, registration renewal, license renewal, replacing a lost title, paying a DRA or scheduling a road test. '

In [ ]:
# training document for sliding windows
td = (document_dataset[250]['doc_text']).split(' ')

start = 0
end = 512
slide = 256
windows = []
doc_size = len(td)
while(start <= doc_size):
    windows.append(td[start:end])
    start += slide
    # if there are less tokens than the slide amount
    if (doc_size - start) < slide:
        end = doc_size - start
    else:
        end += slide        

In [ ]:
for window in windows:
    w = " ".join(window)
    print(w)
    print('----------------------------------------------------------------------------------------------------')